In [2]:
import numpy as np
import pandas as pd
from pandas import json_normalize

from bs4 import BeautifulSoup
import requests 

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.filterwarnings("ignore")

# **Locations**

In [86]:
# pip install geopy

In [87]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="music_analysis")

def get_coordinates(city):
    location = geolocator.geocode(city)
    if location:
        return location.latitude, location.longitude
    return None, None

city = "Paris"
lat, lon = get_coordinates(city)
print(f"{city} -> Latitude: {lat}, Longitude: {lon}")

Paris -> Latitude: 48.8588897, Longitude: 2.3200410217200766


**What if I want Paris, Texas (US) and not the capital of France?**

In [88]:
location = geolocator.geocode("Escàs, España"
                            #   , country_codes="US"
                              , exactly_one=True)

print(location.address)  
print(location.latitude, location.longitude)

Escàs, Rialp, Pallars Sobirà, Lleida, Catalunya, 25567, España
42.4632162 1.1056215


## **Locations dictionary**

In [89]:
df_locations = df_masters_concat.copy()

NameError: name 'df_masters_concat' is not defined

locations_dic = {'Detroit': }

## **Locations Wikipedia scraping**

### Testing

In [3]:
df = pd.read_csv('Datasets/df_ratings.csv')
df.shape

(25481, 5)

In [4]:
df.sample(5)

,album_id,artist,album,rating,votes
6137,10547,k-os,Exit,3.67,32
19101,54469,The New Pornographers,Together,3.36,122
23919,96446,Borknagar,Urd,3.60,173
22361,78724,Signals Midwest,Latitudes & Longitudes,3.98,54
20628,64590,Johnny Cash,The Fabulous Johnny Cash,4.02,63


In [5]:
artists = df['artist'].unique()
len(artists)

7934

In [6]:
artists[0]

'Less Than Jake'

In [7]:
artists[3].replace(' ', '_') + ('_(band)')

'Finch_(band)'

In [8]:
artist = "Sparta"
name_changed = artist.replace(' ', '_')
name_changed_band = artist.replace(' ', '_') + ('_(band)')
url = f"https://en.wikipedia.org/wiki/{name_changed_band}"

response = requests.get(url).content
soup = BeautifulSoup(response, "html.parser")
origin = soup.select('table tr th', class_='infobox-label')[2].text
location = soup.select('table tr td', class_='infobox-data')[1].text
'This article' in location

False

In [9]:
len(soup.select('table tr th', class_='infobox-label'))

56

### Changing values

In [10]:
df[df['artist']=='pg.99 / Majority Rule']

,album_id,artist,album,rating,votes


In [132]:
df['artist'] = np.where(df['artist']=='pg.99 / Majority Rule', 'Majority Rule', df['artist'])
df[df['artist']=='pg.99 / Majority Rule']

,album_id,artist,album,rating,votes


In [134]:
df.to_csv('Datasets/df_ratings.csv', index=False)

### **Code**

In [ ]:
df = pd.read_csv('Datasets/df_ratings.csv')
artists = df['artist'].unique()

artists_list = []
origin_list = []
count=0

for index in artists:
    count+=1
    artists_list.append(index)
    name_changed = index.replace(' ', '_')
    name_changed_band = name_changed + ('_(band)')

    try:
        url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
        response = requests.get(url).content
        soup = BeautifulSoup(response, "html.parser")

        origin = soup.select('table tr th', class_='infobox-label')

        if len(origin) > 0:
            try:
                if origin[2].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[1].text
                elif origin[3].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[2].text

                if 'This article' in location:
                    location = soup.select('table tr td', class_='infobox-data')[3].text        
                    print(f'{count} - {name_changed_band}: {location}')
                    origin_list.append(location)
                else:
                    print(f'{count} - {name_changed_band}: {location}')
                    origin_list.append(location)
            except:
                print(f'{count} - {name_changed_band}: {location}')
                origin_list.append(location)      
        else:
            try:
                url = f"https://en.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")

                origin = soup.select('table tr th', class_='infobox-label')

                if len(origin) > 0:
                    if origin[2].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[1].text

                        if 'This article' in location:
                            location = soup.select('table tr td', class_='infobox-data')[3].text        
                            print(f'{count} - {name_changed_band}: {location}')
                            origin_list.append(location)
                        else:
                            print(f'{count} - {name_changed_band}: {location}')
                            origin_list.append(location)

                    elif origin[3].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[2].text
                        print(f'{count} - {name_changed_band}: {location}')
                        origin_list.append(location) 

                    else:
                        print(f'{count} - {index}: no location found')
                        origin_list.append(np.nan)  
                else:
                    print('{count} - {index}: short length')
                    origin_list.append(np.nan)
            except:
                print('error')
                origin_list.append(np.nan)
    except:
        print('error')
        origin_list.append(np.nan)

1 - Less_Than_Jake_(band): Gainesville, Florida, U.S.
2 - Sparta_(band): El Paso, Texas
3 - Muse_(band): Teignmouth, Devon, England
4 - Finch: no location found
5 - Queens_of_the_Stone_Age_(band): Seattle, Washington, U.S.
6 - Transplants_(band): Los Angeles, California, United States[1]
7 - Dream_Theater_(band): Boston, Massachusetts, United States
8 - A_Perfect_Circle_(band): Los Angeles, California, U.S.
9 - Rooney_(band): Los Angeles, California, U.S.
10 - AFI_(band): Ukiah, California, U.S.
11 - Sublime_(band): Long Beach, California, U.S.
12 - The_Cure_(band): Crawley, England
13 - blink-182_(band): Poway, California, U.S.
14 - Weezer_(band): Los Angeles, California, U.S.
15 - Deftones_(band): Sacramento, California, U.S.
16 - Jimmy_Eat_World_(band): Mesa, Arizona, U.S.
17 - NOFX_(band): Los Angeles, California, U.S.
18 - The_Mars_Volta_(band): El Paso, Texas, U.S.
19 - Black_Sabbath_(band): Birmingham, England
20 - Raised_Fist_(band): Luleå, Norrbotten, Sweden
21 - Yes_(band): L

In [116]:
for index in artists:
    print(index)

Less Than Jake
Sparta
Muse
Finch
Queens of the Stone Age
Transplants
Dream Theater
A Perfect Circle
Rooney
AFI
Sublime
The Cure
blink-182
Weezer
Deftones
Jimmy Eat World
NOFX
The Mars Volta
Black Sabbath
Raised Fist
Yes
Brainiac
Rise Against
Pearl Jam
Joe Satriani
Iron Maiden
Streetlight Manifesto
O.A.R.
The Beatles
Metallica
Incubus (USA-CA)
The Flower Kings
Alice in Chains
Coheed and Cambria
Pantera
Brand New
King Crimson
Tool
Revis
Catch 22
Radiohead
Green Day
The Presidents of the United States of America
Big Black
Filter
Rage Against the Machine
Melvins
The Distillers
Poison the Well
The Network
From Autumn to Ashes
System of a Down
Megadeth
Nine Inch Nails
Pink Floyd
Machine Head
The New Pornographers
Killswitch Engage
In Flames
Regurgitator
Sunny Day Real Estate
Something Corporate
The Smashing Pumpkins
Five Pointe O
...And You Will Know Us by the Trail of Dead
Led Zeppelin
Biohazard
Anti-Flag
The Ataris
Children of Bodom
The Cooper Temple Clause
Audioslave
Fear Factory
The Offs

### **Saving the data into a dataframe**

In [13]:
len(artists_list)

7934

In [14]:
len(origin_list)

7404

In [12]:
df_locations = pd.DataFrame({'artist': artists_list
                             , 'origin': origin_list})
df_locations.shape

ValueError: All arrays must be of the same length

In [ ]:
df_locations.sample(10)

,artist,origin
0,Less Than Jake,"Gainesville, Florida, U.S."
1,Sparta,"El Paso, Texas"
2,Muse,"Teignmouth, Devon, England"
3,Finch,NaN
4,Queens of the Stone Age,"Seattle, Washington, U.S."
5,Transplants,"Los Angeles, California, United States[1]"
6,Dream Theater,"Boston, Massachusetts, United States"
7,A Perfect Circle,"Los Angeles, California, U.S."
8,Rooney,"Los Angeles, California, U.S."
9,AFI,"Ukiah, California, U.S."


### **Analysis and cleaning**

In [ ]:
df_locations['origin'].isna().sum()

In [ ]:
# percentage of nulls
(df_locations['origin'].isna().sum() / df_locations.shape[0])*100

In [ ]:
df_locations.sort_values('origin')

In [ ]:
df_locations['origin'] = df_locations['origin'].str.replace('.', '')
# to avoid problems with the U.S., for instance, the dots cause problems

In [ ]:
df_locations.sort_values('origin')

In [ ]:
df_locations = pd.read_csv('Datasets/df_locations.csv')
locations = df_locations['origin'].unique()

address_list = []
latitude_list = []
longitude_list = []
count = 0

for i in locations[0:10]:
  count+=1
  location = geolocator.geocode(i)

  print(location.address)  
  print(location.latitude, location.longitude)

# save the info in lists
  address_list.append(location.address)
  latitude_list.append(location.latitude)
  longitude_list.append(location.longitude)

## **GeoPy**

In [ ]:
# pip install geopy

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="music_analysis")

def get_coordinates(city):
    location = geolocator.geocode(city)
    if location:
        return location.latitude, location.longitude
    return None, None

city = "Paris"
lat, lon = get_coordinates(city)
print(f"{city} -> Latitude: {lat}, Longitude: {lon}")

Paris -> Latitude: 48.8588897, Longitude: 2.3200410217200766


**What if I want Paris, Texas (US) and not the capital of France?**

In [ ]:
location = geolocator.geocode("Escàs, España"
                            #   , country_codes="US"
                              , exactly_one=True)

print(location.address)  
print(location.latitude, location.longitude)

Escàs, Rialp, Pallars Sobirà, Lleida, Catalunya, 25567, España
42.4632162 1.1056215


### Selenium

In [100]:
url = "https://en.wikipedia.org/wiki/"

PATH = ("C:/Program Files (x86)/chromedriver.exe")
driver = webdriver.Chrome(PATH)
driver.get(url)

time.sleep(2)

search = driver.select('')

NameError: name 'time' is not defined